In [ ]:
from base64 import b64encode, b64decode
from random import randint, getrandbits
from sympy import isprime

def primeGen(k):
    while True:
        p = getrandbits(k)
        if p & 1 == 0:
            p += 1
        if isprime(p):
            return p
        
def RSA_publicKeyRead(filename='RSA_publicKey.txt'):
    with open(filename, 'r') as f:
        eStr = f.readline()
        nStr = f.readline()
        e = int.from_Base64(eStr)
        n = int.from_Base64(nStr)
        
        return e, n
    
def RSA_keygen(k = 1024):
    p = primeGen(k)
    q = primeGen(k)
    
    n = p*q
    e = 65537
    
    d = pow(e, -1, (p-1)*(q-1))
    return p, q, n, e, d

def RSA_encrypt(filename = 'keypub64.txt'):
    e, n = RSA_publicKeyRead(filename)
    strK = input('Message:')
    K = int.from_bytes(strK.encode(), byteorder='little')
    cK = pow(K, e, n)
    ckLen = cK.bit_length() // 8 +1
    cbytes = cK.to_bytes(length = ckLen, byteorder='little')
    return cbytes, len(strK)

def RSA_decrypt(cbytes, ckLen, filename = 'keypriv64.txt'):
    d, p, q = RSA_privateKeyRead(filename)
    n = p*q
    cK = int.from_bytes(cbytes, byteorder='little')
    K1 = pow(cK, d, n)
    message = K1.to_bytes(ckLen, byteorder='little')
    print("Decoded message: ", message.decode)
    

In [ ]:
# 1. feladat

# Válasszunk véletlenszerűen két, legalább 512 bites prímszámot, 
# majd határozzuk meg az RSA publikus és privát kulcsokat. Írjuk ki 
# külön állományba a publikus kulcs base64-es alakját, egy másikba a 
# privát kulcs base64-es alakját. Vegyünk egy tetszőleges ASCII nyomtatható 
# karaktereket tartalmazó szöveget és titkosítsuk valamely állományból 
# beolvasott publikus kulccsal. A titkosított szöveget hexadecimális 
# formában írassuk ki a képernyőre, majd fejtsük vissza a rejtjelezett szöveget. 

import random
import base64
from sympy import isprime, mod_inverse

def generate_large_prime(bits=512):
    """Generate a large prime number of specified bit length."""
    while True:
        num = random.getrandbits(bits)
        if isprime(num):
            return num

def generate_rsa_keys(bits=512):
    """Generate RSA public and private keys."""
    p = generate_large_prime(bits)
    q = generate_large_prime(bits)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Choose e such that 1 < e < phi and gcd(e, phi) = 1
    e = 65537  # Common choice for e
    if phi % e == 0:
        raise ValueError("Regenerate primes; chosen e is not coprime with phi.")
    
    # Calculate d (private key exponent)
    d = mod_inverse(e, phi)
    
    return (e, n), (d, n)

def save_key_to_file(key, filename):
    """Save a key to a file in base64 format."""
    with open(filename, "w") as f:
        key_bytes = str(key).encode('utf-8')
        base64_key = base64.b64encode(key_bytes).decode('utf-8')
        f.write(base64_key)
        
def rsa_encrypt(message, public_key):
    """Encrypt a message with the RSA public key."""
    e, n = public_key
    message_bytes = message.encode('utf-8')
    m = int.from_bytes(message_bytes, byteorder='big')
    c = pow(m, e, n)
    return hex(c)[2:]  # Return as hexadecimal

def rsa_decrypt(ciphertext, private_key):
    """Decrypt a ciphertext with the RSA private key."""
    d, n = private_key
    c = int(ciphertext, 16)  # Convert hex back to integer
    m = pow(c, d, n)
    message_bytes = m.to_bytes((m.bit_length() + 7) // 8, byteorder='big')
    return message_bytes.decode('utf-8')

# Generate RSA keys
public_key, private_key = generate_rsa_keys()

# Save keys to files
save_key_to_file(public_key, "public_key.txt")
save_key_to_file(private_key, "private_key.txt")

# Message to encrypt
message = "This is a secret message."

# Encrypt the message
ciphertext = rsa_encrypt(message, public_key)
print(f"Encrypted message (hex): {ciphertext}")

# Decrypt the message
decrypted_message = rsa_decrypt(ciphertext, private_key)
print(f"Decrypted message: {decrypted_message}")


In [ ]:
# 2. feladat

# Írjunk egy Python programot, amelyben az RSA digitális aláírás algoritmusait implementáljuk:

#   - egy menüpontban, kulcsgenerálás néven lehessen kigenerálni a p, q, e, d, n 
#     értékeket, majd külön állományba lementeni a publikus kulcs base64-es alakját, 
#     és egy másik állományba a privát kulcs base64-es alakját,
#   - egy másik menüpontban lehetőség legyen a fileba lementett privát kulcsot beolvasni 
#     és segítségével egy tetszőleges K értékre meghatározni az aláírás értékét, majd az 
#     értéket és az aláírás base64-es alakját kiírni egy állományba,
#   - egy harmadik menüpontban lehetőség legyen a publikus kulcsot, beolvasni 
#     egy állományból, továbbá egy másik állományból az érték, és az aláírás 
#     base64-es alakját beolvasni majd ellenőrizni az aláírást.

import random
import base64
from sympy import isprime, mod_inverse
from hashlib import sha256

# RSA kulcsgenerálás
def generate_large_prime(bits=512):
    """Generate a large prime number of specified bit length."""
    while True:
        num = random.getrandbits(bits)
        if isprime(num):
            return num

def generate_rsa_keys(bits=512):
    """Generate RSA public and private keys."""
    p = generate_large_prime(bits)
    q = generate_large_prime(bits)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Choose e such that 1 < e < phi and gcd(e, phi) = 1
    e = 65537  # Common choice for e
    if phi % e == 0:
        raise ValueError("Regenerate primes; chosen e is not coprime with phi.")
    
    # Calculate d (private key exponent)
    d = mod_inverse(e, phi)
    
    return (e, n), (d, n)

def save_key_to_file(key, filename):
    """Save a key to a file in base64 format."""
    with open(filename, "w") as f:
        key_bytes = str(key).encode('utf-8')
        base64_key = base64.b64encode(key_bytes).decode('utf-8')
        f.write(base64_key)

# RSA aláírás generálás
def rsa_sign(message, private_key):
    """Sign a message with the RSA private key."""
    d, n = private_key
    # Hash the message using SHA-256
    hash_value = int(sha256(message.encode('utf-8')).hexdigest(), 16)
    # Compute the signature
    signature = pow(hash_value, d, n)
    return signature

# RSA aláírás ellenőrzése
def rsa_verify(message, signature, public_key):
    """Verify a message's signature with the RSA public key."""
    e, n = public_key
    # Hash the message using SHA-256
    hash_value = int(sha256(message.encode('utf-8')).hexdigest(), 16)
    # Decrypt the signature using the public key
    decrypted_signature = pow(signature, e, n)
    return decrypted_signature == hash_value

# Menü választás
def menu():
    while True:
        print("\n1. Kulcsgenerálás")
        print("2. Aláírás generálás")
        print("3. Aláírás ellenőrzése")
        print("4. Kilépés")
        choice = input("Válassz egy opciót: ")
        
        if choice == '1':
            print("Kulcsgenerálás...")
            public_key, private_key = generate_rsa_keys()
            save_key_to_file(public_key, "public_key.txt")
            save_key_to_file(private_key, "private_key.txt")
            print("Kulcsok mentve.")
        
        elif choice == '2':
            print("Aláírás generálás...")
            private_key = load_key_from_file("private_key.txt")
            k_value = input("Add meg a K értéket (tetszőleges szöveg): ")
            signature = rsa_sign(k_value, private_key)
            # Save the signature in base64
            signature_base64 = base64.b64encode(str(signature).encode('utf-8')).decode('utf-8')
            with open("signature.txt", "w") as f:
                f.write(signature_base64)
            print(f"Aláírás mentve: {signature_base64}")
        
        elif choice == '3':
            print("Aláírás ellenőrzése...")
            public_key = load_key_from_file("public_key.txt")
            k_value = input("Add meg a K értéket (tetszőleges szöveg): ")
            with open("signature.txt", "r") as f:
                signature_base64 = f.read()
            signature = int(base64.b64decode(signature_base64).decode('utf-8'))
            if rsa_verify(k_value, signature, public_key):
                print("Az aláírás érvényes.")
            else:
                print("Az aláírás nem érvényes.")
        
        elif choice == '4':
            break
        else:
            print("Érvénytelen választás, próbálj újra!")

# Kulcsok beolvasása fájlból
def load_key_from_file(filename):
    """Load a key from a base64 encoded file."""
    with open(filename, "r") as f:
        key_base64 = f.read()
        key_bytes = base64.b64decode(key_base64.encode('utf-8'))
        return eval(key_bytes.decode('utf-8'))


# Fő program indítása
if __name__ == "__main__":
    menu()


In [ ]:
# 3. feladat

# Ugyanaz a szöveg, háromszor volt rejtjelezve textbook RSA-val, 
# ahol adottak az (3, n1), (3, n2), (3, n3) nyilvános kulcsok. 
# Az n1, n2, n3 értékek a modE3.txt állomány első három sorában találhatóak, 
# ilyen sorrendben. A megfelelő rejtjelezett szövegek az cryptE3_1, cryptE3_2, 
# cryptE3_3 állományokban találhatóak. Határozzuk meg az eredeti szöveget.

import math
import decimal
from sympy.ntheory.modular import solve_congruence

# Kínai maradéktétel használata
def chinese_remainder_theorem(congruences):
    """Solve the system of congruences using the Chinese Remainder Theorem."""
    result, _ = solve_congruence(*congruences)
    return result

# Köbgyök kiszámítása a dekódolt szám alapján
def cubic_root(x):
    """Return the cubic root of a number rounded to the nearest integer."""
    decimal.getcontext().prec = 100  # Set precision to avoid floating point errors
    return math.ceil(pow(x, 1/decimal.Decimal(3)))

# Beolvassuk az RSA titkosított üzeneteket és a modulusokat
def read_files():
    """Read the moduli and encrypted texts from the files."""
    with open("modE3.txt", "r") as mod_file:
        n1 = int(mod_file.readline().strip())
        n2 = int(mod_file.readline().strip())
        n3 = int(mod_file.readline().strip())

    with open("cryptE3_1", "rb") as c1_file:
        c1 = int.from_bytes(c1_file.read(), byteorder='big')

    with open("cryptE3_2", "rb") as c2_file:
        c2 = int.from_bytes(c2_file.read(), byteorder='big')

    with open("cryptE3_3", "rb") as c3_file:
        c3 = int.from_bytes(c3_file.read(), byteorder='big')

    return (n1, n2, n3), (c1, c2, c3)

# Fő program
def main():
    # Beolvassuk a fájlokat
    (n1, n2, n3), (c1, c2, c3) = read_files()

    # Kínai maradéktétel alkalmazása
    congruences = [
        (c1, n1),
        (c2, n2),
        (c3, n3)
    ]
    x = chinese_remainder_theorem(congruences)

    # Köbgyök vonása
    original_number = cubic_root(x)

    # Eredeti szöveg visszaalakítása
    original_text = bytearray()
    while original_number > 0:
        original_text.insert(0, original_number & 0xFF)
        original_number >>= 8

    print("Eredeti szöveg:", original_text.decode('utf-8'))

if __name__ == "__main__":
    main()


In [ ]:
# 4. feladat

# Írjunk Python programot, amely meghatározza a következő r egyenletből 
# álló kongruencia-rendszer legkisebb megoldását, feltételezve, hogy az 
# m1, m2,…, mr pozitív egész számok páronként relatív prímek.

#   x = a1 mod m1
#   x = a2 mod m2
#   ...
#   x = ar mod mr 

def egcd(a, b):
    """Extended Euclidean Algorithm to find the GCD and the coefficients (x, y) such that a * x + b * y = gcd(a, b)."""
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = egcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def mod_inverse(a, m):
    """Returns the modular inverse of a under modulo m using the Extended Euclidean Algorithm."""
    gcd, x, _ = egcd(a, m)
    if gcd != 1:
        raise ValueError(f"The modular inverse of {a} modulo {m} does not exist.")
    return x % m

def chinese_remainder_theorem(a_values, m_values):
    """Solve the system of congruences using the Chinese Remainder Theorem."""
    # Step 1: Calculate the product of all moduli (M)
    M = 1
    for m in m_values:
        M *= m

    # Step 2: Calculate the solution using the formula:
    # x = sum(a_i * M_i * y_i) for i = 1 to r, where M_i = M / m_i, and y_i is the modular inverse of M_i mod m_i.
    x = 0
    for a_i, m_i in zip(a_values, m_values):
        M_i = M // m_i
        y_i = mod_inverse(M_i, m_i)  # Find modular inverse of M_i mod m_i
        x += a_i * M_i * y_i

    # Step 3: Return the solution modulo M
    return x % M

# Example usage:
def main():
    # Example system of congruences:
    # x ≡ 2 (mod 3)
    # x ≡ 3 (mod 5)
    # x ≡ 1 (mod 7)
    
    a_values = [2, 3, 1]  # The remainders (a1, a2, a3)
    m_values = [3, 5, 7]  # The moduli (m1, m2, m3)

    solution = chinese_remainder_theorem(a_values, m_values)
    print("The smallest solution to the system of congruences is:", solution)

if __name__ == "__main__":
    main()


In [ ]:
# 5. feladat

# Az RSA-textbook kriptorendszer visszafejtésének időigényét gyorsítani lehet, 
# ha alkalmazzuk a kínai maradéktételt. Írjunk Python programot, amelyben 
# összehasonlítjuk a két visszafejtési algoritmus időigényét. 

import time
import random
from sympy import isprime

# RSA-textbook visszafejtés
def rsa_decrypt_textbook(c, d, n):
    return pow(c, d, n)

# Kínai maradéktétel visszafejtés
def chinese_remainder_theorem(a, p, q):
    # Apply Chinese Remainder Theorem
    n = p * q
    p_inv = mod_inverse(p, q)
    q_inv = mod_inverse(q, p)

    x = (a[0] * q * q_inv + a[1] * p * p_inv) % n
    return x

def mod_inverse(a, m):
    """Find the modular inverse of a under modulo m."""
    return pow(a, m - 2, m)

def rsa_decrypt_crt(c, d, p, q):
    # Decrypt using CRT
    dp = d % (p - 1)
    dq = d % (q - 1)
    m1 = pow(c, dp, p)
    m2 = pow(c, dq, q)

    return chinese_remainder_theorem([m1, m2], p, q)

def generate_rsa_keys(bits=512):
    # Generate random primes p and q
    p = random.getrandbits(bits)
    while not isprime(p):
        p = random.getrandbits(bits)
    
    q = random.getrandbits(bits)
    while not isprime(q):
        q = random.getrandbits(bits)
    
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Choose e
    e = 65537
    
    # Compute d (modular inverse of e mod phi)
    d = mod_inverse(e, phi)
    
    return p, q, e, d, n

def main():
    # Test parameters
    bits = 512  # 512 bit RSA for demonstration
    
    # Generate RSA keys
    p, q, e, d, n = generate_rsa_keys(bits)
    
    # Message to encrypt
    m = random.randint(0, n)
    
    # Encrypt message
    c = pow(m, e, n)
    
    # Measure RSA-textbook decryption time
    start_time = time.time()
    decrypted_message_textbook = rsa_decrypt_textbook(c, d, n)
    end_time = time.time()
    rsa_textbook_time = end_time - start_time
    
    # Measure RSA-CRT decryption time
    start_time = time.time()
    decrypted_message_crt = rsa_decrypt_crt(c, d, p, q)
    end_time = time.time()
    rsa_crt_time = end_time - start_time
    
    # Results
    print(f"Original message: {m}")
    print(f"Decrypted message (Textbook): {decrypted_message_textbook}")
    print(f"Decrypted message (CRT): {decrypted_message_crt}")
    print(f"RSA-textbook decryption time: {rsa_textbook_time:.6f} seconds")
    print(f"RSA-CRT decryption time: {rsa_crt_time:.6f} seconds")
    
if __name__ == "__main__":
    main()


In [ ]:
# 6. feladat

# Írjunk függvényt, amely meghatározza két "nagy" szám: X, Y összegét. 
# Feltételezzük, hogy a gépszó hosszúsága nem nagyobb mint, H és X, Y 
# egyike sem nagyobb, mint M, ahol természetesen H jóval kisebb, 
# mint M. Pédául ha H = 100, akkor M = 106, egy lehetséges választás

# Először kiválasztjuk az m1, m2,..., mk értékeket, úgy hogy m1 · m2· ...· mk >= M, 
# és m1, m2, ..., mk mindegyike kisebb legyen, mint H, és páronként relatív 
# prímek legyenek, ahol az m1, m2,..., mk értékeket választhajuk prímeknek is. 

# Meghatározzuk a következő kongruenciákat:
#   x1 = X mod m1, y1 = Y mod m1
#   x2 = X mod m2, y2 = Y mod m2
#   ...
#   xk = X mod mk, yk = Y mod mk

# majd a Kínai maradéktétellel megoldjuk a következő rendszert, ahol a 
# kapott z érték, az X és Y összegét fogja jelenteni:
#   z = x1 + y1 mod m1,
#   z = x2 + y2 mod m2,
#   ...
#   z = xk + yk mod mk,
# Határozzuk meg abban a sajátos esetben is, a két "nagy" szám: X, Y összegét, 
# ha m1 = 2i1-1, m2 = 2i2-1,..., mk = 2ik-1, ahol k egy pozitív egész szám. 

import math
from sympy import isprime
from functools import reduce

# Kínai maradéktétel (Chinese Remainder Theorem)
def chinese_remainder_theorem(a, m):
    """
    Megoldja a következő kongruenciát:
    x ≡ a_i (mod m_i) minden i-ra
    a: lista az a_i értékekkel
    m: lista az m_i értékekkel
    """
    total_product = reduce(lambda x, y: x * y, m)
    result = 0
    for ai, mi in zip(a, m):
        mi_product = total_product // mi
        mi_inverse = mod_inverse(mi_product, mi)
        result += ai * mi_product * mi_inverse
    return result % total_product

# Moduláris inverz
def mod_inverse(a, m):
    """Euklidészi algoritmus segítségével meghatározza a moduláris inverzét"""
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

# A számok összegének meghatározása a kínai maradéktétel segítségével
def sum_large_numbers(X, Y, H, M):
    # Kiválasztjuk a megfelelő modulusokat
    k = 1
    m_values = []
    while True:
        m_i = 2**k - 1
        if m_i >= M:
            break
        m_values.append(m_i)
        k += 1

    # Kongruenciák meghatározása
    x_values = [X % m for m in m_values]
    y_values = [Y % m for m in m_values]

    # A kongruenciák szerint oldjuk meg a rendszert a kínai maradéktétellel
    z = chinese_remainder_theorem([x + y for x, y in zip(x_values, y_values)], m_values)

    return z

# Tesztpélda
H = 100
M = 106
X = 1234567890123456789012345678901234567890
Y = 9876543210987654321098765432109876543210

result = sum_large_numbers(X, Y, H, M)
print(f"A két szám összege: {result}")
